In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

dataset = pd.read_csv("dublinbikes_20200101_20200401.csv")
df = dataset

df1 = df.copy()
df1['STATUS'] = df1['STATUS'].astype('category')
df1['LAST UPDATED'] = pd.to_datetime(df1['LAST UPDATED'])
df1['TIME'] = pd.to_datetime(df1['TIME'])
df1['formatted_time'] = df1['TIME'].dt.floor('h')

df1['day_of_week'] = df1['formatted_time'].dt.strftime('%A')
df1['day_of_month'] = df1['formatted_time'].dt.strftime('%d').astype(np.int64)
df1['hour'] = df1['formatted_time'].dt.strftime('%H').astype(np.int64)
df1['month'] = df1['formatted_time'].dt.strftime('%m').astype(np.int64)
df1['week'] = df1['formatted_time'].dt.strftime('%w').astype(np.int64)
numeric_columns = df1.select_dtypes(['int64']).columns
print(df1[numeric_columns].describe().T)
print(df1.select_dtypes(['category']).describe().T)
print(df1.dtypes)

ts = pd.to_datetime('2020/02/01')
te = pd.to_datetime('2020/04/01')
mask = (df1['TIME'] >= ts) & (df1['TIME'] <= te)
pd.options.mode.chained_assignment = None
#df['TIME'].max() - df['TIME'].min()
suburb_point = "Merrion Square South"
#suburb_point = "Grangegorman Lower (South)"
suburb_df = df1.loc[mask]
suburb_df = suburb_df[suburb_df['ADDRESS'] == suburb_point]
suburb_dataset = suburb_df[['TIME', 'AVAILABLE BIKES']]
#fig, ax = plt.subplots()

suburb_dataset['date'] = suburb_dataset['TIME'].dt.floor('T')
suburb_dataset = suburb_dataset.reset_index()
suburb_dataset.drop('TIME',axis=1, inplace=True)
time = suburb_dataset['date']
bikes = suburb_dataset['AVAILABLE BIKES']
#plot 2 week data for Merrion Square South region
fig = plt.figure()
ax = fig.add_subplot(111)
ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax.scatter(time,bikes, color="red", marker=".", label='bike count')
ax.set_xlabel("date")
ax.set_ylabel("bike available")
fig.autofmt_xdate()
ax.grid(True)
ax.set_title(f'Bike usage for {suburb_point}')
plt.show()

                           count       mean        std   min   25%   50%  \
STATION ID             2228278.0  60.196628  33.598271   2.0  31.0  61.0   
BIKE STANDS            2228278.0  32.184342   7.666891  16.0  29.0  30.0   
AVAILABLE BIKE STANDS  2228278.0  20.456110  11.140868   0.0  12.0  20.0   
AVAILABLE BIKES        2228278.0  11.644086   9.980968   0.0   3.0  10.0   
day_of_month           2228278.0  15.902037   9.262035   1.0   8.0  16.0   
hour                   2228278.0  11.556068   6.876381   0.0   6.0  12.0   
month                  2228278.0   2.317890   0.729879   1.0   2.0   2.0   
week                   2228278.0   2.991054   1.976522   0.0   1.0   3.0   

                        75%    max  
STATION ID             90.0  117.0  
BIKE STANDS            40.0   40.0  
AVAILABLE BIKE STANDS  29.0   41.0  
AVAILABLE BIKES        18.0   40.0  
day_of_month           24.0   31.0  
hour                   17.0   23.0  
month                   3.0    4.0  
week               

C:\Users\rkhna\AppData\Local\Temp/ipykernel_7940/2053719420.py:51: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  fig.autofmt_xdate()


In [17]:
from sklearn.metrics import r2_score
import math
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

day_of_month = suburb_df['day_of_month']
hour = suburb_df['hour']

t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

q=2
lag_range = [1,2,3,4]
stride=1
Ci = 50
scores = []
errors = []
for lag in lag_range:
    w=math.floor(7*24*60*60/dt)
    length = bikes.size - w - lag * w - q
    print("intial set : ", bikes.size, w, lag, q, length)
    XX=bikes[q:q+length:stride]
    X1 = day_of_month[q:q+length:stride]
    X2 = hour[q:q+length:stride]
    XX=np.column_stack((XX,X1, X2))
    #week
    for i in range(1,lag):
        X=bikes[i*w+q:i*w+q+length:stride]
        X1 = day_of_month[i*w+q:i*w+q+length:stride]
        X2 = hour[i*w+q:i*w+q+length:stride]
        XX=np.column_stack((XX,X, X1, X2))
    d=math.floor(24*60*60/dt)
    #days
    for i in range(0,lag):
        X=bikes[i*d+q:i*d+q+length:stride]
        X1=day_of_month[i*d+q:i*d+q+length:stride]
        X2=hour[i*d+q:i*d+q+length:stride]
        XX=np.column_stack((XX,X, X1, X2))

    for i in range(0,lag):
        X=bikes[i:i+length:stride]
        X1=day_of_month[i:i+length:stride]
        X2=hour[i:i+length:stride]
        XX=np.column_stack((XX,X, X1, X2))

    yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
    tt=time[lag*w+w+q:lag*w+w+q+length:stride]
    
    yy.reset_index(drop=True, inplace=True)
    tt.reset_index(drop=True, inplace=True)

    train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

    a = 1/(2*Ci)
    print("lag value - ", lag)
    model = Lasso(alpha= a).fit(XX[train], yy[train])
    y_pred = model.predict(XX)
    score = model.score(XX[test], yy[test])
    from sklearn.model_selection import cross_val_score
    mse = mean_squared_error(yy,y_pred)
    errors.append(mse)
    r2 = r2_score(yy,y_pred)
    scores.append(r2)
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}"
    .format(score,  mse,np.sqrt(mse)))
    print(f'r2_score - {r2}')

plt.close("all")
fig, (ax1, ax2) = plt.subplots(1, 2)
#fig=plt.figure()
#ax1=fig.add_subplot(111)
ax1.plot(lag_range,scores)
ax1.set_xlabel('Alpha (lag range)')
ax1.set_ylabel('Beta (Score)')
ax1.set_title('Model Score vs Lag Value')
ax1.axis('tight')

# fig=plt.figure()
# ax2=fig.add_subplot(111)

ax2 = plt.gca()
ax2.ticklabel_format(useOffset=False)
errors = [round(num, 5) for num in errors]

ax2.plot(lag_range, errors)
ax2.set_xlabel("lag")
ax2.set_ylabel("error")
ax2.set_title("Error vs Lag")
ax2.axis("tight")
fig.show()
# ax2.set_yticklabels(errors)
print(scores)
print(errors)
print(lag_range)

C:\Users\rkhna\AppData\Local\Temp/ipykernel_3884/144600625.py:11: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


intial set :  17127 2016 1 2 13093
lag value -  1
SCORE: 0.095, MSE:36.11, RMSE:6.01
r2_score - 0.09683234583070288
intial set :  17127 2016 2 2 11077
lag value -  2
SCORE: 0.303, MSE:29.11, RMSE:5.40
r2_score - 0.3141160035928825
intial set :  17127 2016 3 2 9061
lag value -  3
SCORE: 0.555, MSE:20.20, RMSE:4.49
r2_score - 0.5536572873652599
intial set :  17127 2016 4 2 7045
lag value -  4


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 28442.756388018955, tolerance: 32.89392437913912
  model = cd_fast.enet_coordinate_descent(


SCORE: 0.530, MSE:23.78, RMSE:4.88
r2_score - 0.5275314079057982
[0.09683234583070288, 0.3141160035928825, 0.5536572873652599, 0.5275314079057982]
[36.11213, 29.11242, 20.20128, 23.77683]
[1, 2, 3, 4]


In [18]:
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import math

%matplotlib qt

t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

day_of_month = suburb_df['day_of_month']
hour = suburb_df['hour']

q=2
lag=3
stride=1
w=math.floor(7*24*60*60/dt)
length = bikes.size - w - lag * w - q
XX=bikes[q:q+length:stride]
X1 = day_of_month[q:q+length:stride]
X2 = hour[q:q+length:stride]
XX=np.column_stack((XX,X1, X2))
#week
for i in range(1,lag):
    X=bikes[i*w+q:i*w+q+length:stride]
    X1 = day_of_month[i*w+q:i*w+q+length:stride]
    X2 = hour[i*w+q:i*w+q+length:stride]
    XX=np.column_stack((XX,X, X1, X2))
d=math.floor(24*60*60/dt)
#days
for i in range(0,lag):
    X=bikes[i*d+q:i*d+q+length:stride]
    X1=day_of_month[i*d+q:i*d+q+length:stride]
    X2=hour[i*d+q:i*d+q+length:stride]
    XX=np.column_stack((XX,X, X1, X2))

for i in range(0,lag):
    X=bikes[i:i+length:stride]
    X1=day_of_month[i:i+length:stride]
    X2=hour[i:i+length:stride]
    XX=np.column_stack((XX,X, X1, X2))

yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
tt=time[lag*w+w+q:lag*w+w+q+length:stride]

yy.reset_index(drop=True, inplace=True)
tt.reset_index(drop=True, inplace=True)


train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

coeff = []
alphas = []
errors = []
scores = []
Ci = 10
qrange = [1, 2, 3, 4]


c_temp = []
mean_error = []
std_error = []
for q in qrange:
    print(q)
    temp = []
    XPoly = PolynomialFeatures(q).fit_transform(XX)
    a = 1/(2*Ci)
    print(Ci, a)
    model = Lasso(alpha= a).fit(XPoly[train], yy[train])
    coeff.append(model.coef_)
    alphas.append(a)
    y_pred = model.predict(XPoly)
    score = model.score(XPoly[test], yy[test])
    #r2_score = r2_score(yy,y_pred)
    # scores = cross_val_score(model, XX[test], yy[test], cv=5, scoring='f1')
    mse = mean_squared_error(yy,y_pred)
    errors.append(np.sqrt(mse))
    r2 = r2_score(yy,y_pred)
    scores.append(r2)
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}"
   .format(score,  mse,np.sqrt(mse)))
    #print(r2_score(yy,y_pred))
    print(f'r2_score - {r2}')


plt.close("all")
fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.plot(qrange,scores)
ax1.set_xlabel('Alpha (Polynomial range)')
ax1.set_ylabel('Beta (Score)')
ax1.set_title('Model Score vs Polynomial Value')
ax1.axis('tight')

ax2 = plt.gca()
ax2.ticklabel_format(useOffset=False)
errors = [round(num, 5) for num in errors]
ax2.plot(qrange, errors)
ax2.set_xlabel("ploynomial value")
ax2.set_ylabel("error")
ax2.set_title("Error vs polynomial")
ax2.axis("tight")
fig.show()

plt.show()

C:\Users\rkhna\AppData\Local\Temp/ipykernel_3884/2814713869.py:11: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


1
10 0.05
SCORE: 0.548, MSE:20.20, RMSE:4.49
r2_score - 0.5536864452511905
2
10 0.05


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 34069.99528123437, tolerance: 32.394982726269276
  model = cd_fast.enet_coordinate_descent(


SCORE: 0.778, MSE:10.04, RMSE:3.17
r2_score - 0.7781633236608934
3
10 0.05


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18094.197954347608, tolerance: 32.394982726269276
  model = cd_fast.enet_coordinate_descent(


SCORE: 0.889, MSE:4.91, RMSE:2.22
r2_score - 0.8914390893181029
4
10 0.05


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8721.022245747445, tolerance: 32.394982726269276
  model = cd_fast.enet_coordinate_descent(


SCORE: 0.944, MSE:2.36, RMSE:1.54
r2_score - 0.9478441574854815


In [38]:
# find out C

from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection  import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import math

t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

day_of_month = suburb_df['day_of_month']
hour = suburb_df['hour']

q=2
lag=3
stride=1
w=math.floor(7*24*60*60/dt)
length = bikes.size - w - lag * w - q
XX=bikes[q:q+length:stride]
X1 = day_of_month[q:q+length:stride]
X2 = hour[q:q+length:stride]
XX=np.column_stack((XX,X1, X2))
#week
for i in range(1,lag):
    X=bikes[i*w+q:i*w+q+length:stride]
    X1 = day_of_month[i*w+q:i*w+q+length:stride]
    X2 = hour[i*w+q:i*w+q+length:stride]
    XX=np.column_stack((XX,X, X1, X2))
d=math.floor(24*60*60/dt)
#days
for i in range(0,lag):
    X=bikes[i*d+q:i*d+q+length:stride]
    X1=day_of_month[i*d+q:i*d+q+length:stride]
    X2=hour[i*d+q:i*d+q+length:stride]
    XX=np.column_stack((XX,X, X1, X2))

for i in range(0,lag):
    X=bikes[i:i+length:stride]
    X1=day_of_month[i:i+length:stride]
    X2=hour[i:i+length:stride]
    XX=np.column_stack((XX,X, X1, X2))

yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
tt=time[lag*w+w+q:lag*w+w+q+length:stride]

yy.reset_index(drop=True, inplace=True)
tt.reset_index(drop=True, inplace=True)

train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

coeff = []
alphas = []
errors = []
c_mean_error = []
c_std_error = []

C = [0.01, 0.1, 1, 3, 5, 10, 50,  100]

for Ci in C:
    c_temp = []
    mean_error = []
    std_error = []
    XPoly = PolynomialFeatures(4).fit_transform(XX)
    a = 1/(2*Ci)
    print(Ci, a)
    model = Lasso(alpha= a).fit(XPoly[train], yy[train])
    print("coefficient " ,model.intercept_, model.coef_)
    coeff.append(model.coef_)
    alphas.append(a)
    y_pred = model.predict(XPoly)
    score = model.score(XPoly[test], yy[test])
    cross_val = cross_val_score(model, XPoly[test], yy[test], cv=10)
    c_mean_error.append(np.mean(cross_val))
    c_std_error.append(np.std(cross_val))
    # scores = cross_val_score(model, XX[test], yy[test], cv=5, scoring='f1')
    mse = mean_squared_error(yy,y_pred)
    errors.append(np.sqrt(mse))
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}"
   .format(score,  mse,np.sqrt(mse)))
    print(f'r2_score - {r2_score(yy,y_pred)}')

plt.close("all")
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
#fig=plt.figure()
#ax1=fig.add_subplot(111)
ax1.plot(alphas,coeff)
ax1.set_xlabel('Alpha (Regularization Parameter)')
ax1.set_ylabel('Beta (Predictor Coefficients)')
ax1.set_title('Ridge Coefficients vs Regularization Parameters')
ax1.axis('tight')

ax2.errorbar(C, c_mean_error, yerr=c_std_error)
#ax2.set_xticks(C)
ax2.set_xlabel("ploynomial value")
ax2.set_ylabel("error")
ax2.set_title("Error vs polynomial")
ax2.axis("tight")

ax3 = plt.gca()
ax3.ticklabel_format(useOffset=False)
errors = [round(num, 5) for num in errors]
ax3.plot(alphas, errors)
ax3.set_xlabel("alpha")
ax3.set_ylabel("RMSE error")
ax3.set_title("Coefficient error as a function of the regularization")
ax3.axis("tight")
fig.show()

C:\Users\rkhna\AppData\Local\Temp/ipykernel_7940/2480376473.py:13: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


0.01 50.0


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14942.639442909025, tolerance: 32.922124448123604
  model = cd_fast.enet_coordinate_descent(


coefficient  4.5325792433734815 [ 0. -0.  0. ... -0. -0.  0.]


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2283.4344347852975, tolerance: 7.356538810545679
  model = cd_fast.enet_coordinate_descent(
C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2121.8329002968067, tolerance: 7.307520539546292
  model = cd_fast.enet_coordinate_descent(
C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective 

...

SCORE: 0.938, MSE:2.35, RMSE:1.53
r2_score - 0.9481144540824624
100 0.005


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8220.899662259204, tolerance: 32.922124448123604
  model = cd_fast.enet_coordinate_descent(


coefficient  6.973945444046467 [ 0.00000000e+00 -0.00000000e+00  3.10796318e-01 ...  1.04615455e-07
  3.47439569e-07 -1.02703124e-06]


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1174.459449878104, tolerance: 7.356538810545679
  model = cd_fast.enet_coordinate_descent(
C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1156.4698070528475, tolerance: 7.307520539546292
  model = cd_fast.enet_coordinate_descent(
C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective d

SCORE: 0.938, MSE:2.35, RMSE:1.53
r2_score - 0.9481349184513405


In [39]:
fig, ax2 = plt.subplots()
ax2.errorbar(C, c_mean_error, yerr=c_std_error)
#ax2.set_xticks(C)
ax2.set_xlabel("ploynomial value")
ax2.set_ylabel("error")
ax2.set_title("Merrion Error vs polynomial")
ax2.axis("tight")
fig.show()

In [28]:
import math
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

#%matplotlib inline
%matplotlib qt

t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

day_of_month = suburb_df['day_of_month']
hour = suburb_df['hour']

def get_feature_data(q, lag, stride, bikes, time, dt):
    w=math.floor(7*24*60*60/dt)
    length = bikes.size - w - lag * w - q
    XX=bikes[q:q+length:stride]
    X1 = day_of_month[q:q+length:stride]
    X2 = hour[q:q+length:stride]
    XX=np.column_stack((XX,X1, X2))
    #week
    for i in range(1,lag):
        X=bikes[i*w+q:i*w+q+length:stride]
        X1 = day_of_month[i*w+q:i*w+q+length:stride]
        X2 = hour[i*w+q:i*w+q+length:stride]
        XX=np.column_stack((XX,X, X1, X2))
    d=math.floor(24*60*60/dt)
    #days
    for i in range(0,lag):
        X=bikes[i*d+q:i*d+q+length:stride]
        X1=day_of_month[i*d+q:i*d+q+length:stride]
        X2=hour[i*d+q:i*d+q+length:stride]
        XX=np.column_stack((XX,X, X1, X2))

    for i in range(0,lag):
        X=bikes[i:i+length:stride]
        X1=day_of_month[i:i+length:stride]
        X2=hour[i:i+length:stride]
        XX=np.column_stack((XX,X, X1, X2))

    yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
    tt=time[lag*w+w+q:lag*w+w+q+length:stride]

    yy.reset_index(drop=True, inplace=True)
    tt.reset_index(drop=True, inplace=True)
    return XX, yy, tt


def run_model(q, lag, stride, Ci, poly):
    plt.close("all")
    fig, ax = plt.subplots()
    XX, yy, tt = get_feature_data(q, lag, stride, bikes, time, dt)
    train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

    XPoly = PolynomialFeatures(poly).fit_transform(XX)
    a = 1/(2*Ci)
    print(f'Alpha & Ci  Value - {a} {Ci}')
    print(f"q value - {q}")
    model = Lasso(alpha= a).fit(XPoly[train], yy[train])
    y_pred = model.predict(XPoly)
    score = model.score(XPoly[test], yy[test])
    #r2_score = r2_score(yy,y_pred)
    mse = mean_squared_error(yy,y_pred)
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}" .format(score,  mse,np.sqrt(mse)))
    r2 = r2_score(yy,y_pred)
    print(f'r2_score : {r2}\n')
    ax.scatter(time, bikes, color="blue")
    ax.scatter(tt, y_pred, color="green") 
    ax.set_xlabel("time (days)")
    ax.set_ylabel("#bikes") 
    ax.set_title(f"Ridge model for q = {q}")
    ax.legend(["training data","predictions"],loc="upper right") 
    ax.set_ylim([-10, 45])
    fig.show()

q = 12
lag = 3
stride = 1
Ci = 50
poly = 4
run_model(q, lag, stride, Ci, poly)

C:\Users\rkhna\AppData\Local\Temp/ipykernel_7940/2445253717.py:12: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


Alpha & Ci  Value - 0.01 50
q value - 12


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6882.267312067216, tolerance: 32.659037348066256
  model = cd_fast.enet_coordinate_descent(


SCORE: 0.946, MSE:1.98, RMSE:1.41
r2_score : 0.956320069444668



In [42]:
#final run with all identified parameters

import math
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

#%matplotlib inline
%matplotlib qt


t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

def get_feature_data(q, lag, stride, bikes, time, dt):
    w=math.floor(7*24*60*60/dt)
    length = bikes.size - w - lag * w - q
    XX=bikes[q:q+length:stride]
    X1 = day_of_month[q:q+length:stride]
    X2 = hour[q:q+length:stride]
    XX=np.column_stack((XX,X1, X2))
    #week
    for i in range(1,lag):
        X=bikes[i*w+q:i*w+q+length:stride]
        X1 = day_of_month[i*w+q:i*w+q+length:stride]
        X2 = hour[i*w+q:i*w+q+length:stride]
        XX=np.column_stack((XX,X, X1, X2))
    d=math.floor(24*60*60/dt)
    #days
    for i in range(0,lag):
        X=bikes[i*d+q:i*d+q+length:stride]
        X1=day_of_month[i*d+q:i*d+q+length:stride]
        X2=hour[i*d+q:i*d+q+length:stride]
        XX=np.column_stack((XX,X, X1, X2))

    for i in range(0,lag):
        X=bikes[i:i+length:stride]
        X1=day_of_month[i:i+length:stride]
        X2=hour[i:i+length:stride]
        XX=np.column_stack((XX,X, X1, X2))

    yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
    tt=time[lag*w+w+q:lag*w+w+q+length:stride]

    yy.reset_index(drop=True, inplace=True)
    tt.reset_index(drop=True, inplace=True)
    return XX, yy, tt

qrange = [ 2, 6, 12]
lag = 3
stride = 1
plt.close("all")
fig, ax = plt.subplots(3)
ax_index = 0
Ci = 50
poly = 4
q_score = {}
xlim = -5
ylim = 35 # 40 for Grangegorman
for q in qrange:
    XX, yy, tt = get_feature_data(q, lag, stride, bikes, time, dt)
    train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

    XPoly = PolynomialFeatures(poly).fit_transform(XX)
    a = 1/(2*Ci)
    print(f'Alpha & Ci  Value - {a} {Ci}')
    print(f"q value - {q}")
    model = Lasso(alpha= a).fit(XPoly[train], yy[train])
    y_pred = model.predict(XPoly)
    score = model.score(XPoly[test], yy[test])
    #r2_score = r2_score(yy,y_pred)
    mse = mean_squared_error(yy,y_pred)
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}" .format(score,  mse,np.sqrt(mse)))
    r2 = r2_score(yy,y_pred)
    print(f'r2_score - {r2}\n')
    q_score[q] = r2

#     plt.close("all")
#     fig, ax = plt.subplots()

    ax[ax_index].scatter(time, bikes, color="blue")
    ax[ax_index].scatter(tt, y_pred, color="green") 
    ax[ax_index].set_xlabel("time (days)")
    ax[ax_index].set_ylabel("#bikes") 
    ax[ax_index].set_title(f"Merrion Square - Lasso model for q = {q}")
    ax[ax_index].legend(["training data","predictions"],loc="upper right")
    ax[ax_index].set_ylim([xlim, ylim])
    ax_index += 1
print(({"-"*30}))
for q, score in q_score.items():
    print(f"q value {q} - score {score}")
fig.show()

C:\Users\rkhna\AppData\Local\Temp/ipykernel_7940/4189380681.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


Alpha & Ci  Value - 0.01 50
q value - 2


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15168.323019498359, tolerance: 57.1932217301325
  model = cd_fast.enet_coordinate_descent(


SCORE: 0.933, MSE:4.33, RMSE:2.08
r2_score - 0.9450485153539341

Alpha & Ci  Value - 0.01 50
q value - 6


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14163.245971527123, tolerance: 57.324457336093815
  model = cd_fast.enet_coordinate_descent(


SCORE: 0.935, MSE:4.07, RMSE:2.02
r2_score - 0.9483532846242023

Alpha & Ci  Value - 0.01 50
q value - 12


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13358.0458344858, tolerance: 56.52356407458562
  model = cd_fast.enet_coordinate_descent(


SCORE: 0.953, MSE:3.67, RMSE:1.92
r2_score - 0.9534621005568733

{'------------------------------'}
q value 2 - score 0.9450485153539341
q value 6 - score 0.9483532846242023
q value 12 - score 0.9534621005568733


In [15]:
#### CROSS VALIDATION


from sklearn.model_selection  import cross_val_score

from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import math

%matplotlib qt

t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

day_of_month = suburb_df['day_of_month']
hour = suburb_df['hour']

q=2
lag=3
stride=1
w=math.floor(7*24*60*60/dt)
length = bikes.size - w - lag * w - q
XX=bikes[q:q+length:stride]
X1 = day_of_month[q:q+length:stride]
X2 = hour[q:q+length:stride]
XX=np.column_stack((XX,X1, X2))
#week
for i in range(1,lag):
    X=bikes[i*w+q:i*w+q+length:stride]
    X1 = day_of_month[i*w+q:i*w+q+length:stride]
    X2 = hour[i*w+q:i*w+q+length:stride]
    XX=np.column_stack((XX,X, X1, X2))
d=math.floor(24*60*60/dt)
#days
for i in range(0,lag):
    X=bikes[i*d+q:i*d+q+length:stride]
    X1=day_of_month[i*d+q:i*d+q+length:stride]
    X2=hour[i*d+q:i*d+q+length:stride]
    XX=np.column_stack((XX,X, X1, X2))

for i in range(0,lag):
    X=bikes[i:i+length:stride]
    X1=day_of_month[i:i+length:stride]
    X2=hour[i:i+length:stride]
    XX=np.column_stack((XX,X, X1, X2))

yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
tt=time[lag*w+w+q:lag*w+w+q+length:stride]

yy.reset_index(drop=True, inplace=True)
tt.reset_index(drop=True, inplace=True)


train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

coeff = []
alphas = []
errors = []
scores = []
Ci = 10
qrange = [1, 2, 3, 4]


c_temp = []
mean_error = []
std_error = []
for q in qrange:
    print(q)
    temp = []
    XPoly = PolynomialFeatures(q).fit_transform(XX)
    a = 1/(2*Ci)
    print(Ci, a)
    model = Lasso(alpha= a,tol=1e-2).fit(XPoly[train], yy[train])
    coeff.append(model.coef_)
    alphas.append(a)
    y_pred = model.predict(XPoly)
    score = model.score(XPoly[test], yy[test])
    cross_val = cross_val_score(model, XPoly[test], yy[test], cv=10)
    mean_error.append(np.mean(cross_val))
    std_error.append(np.std(cross_val))
    #r2_score = r2_score(yy,y_pred)
    # scores = cross_val_score(model, XX[test], yy[test], cv=5, scoring='f1')
    mse = mean_squared_error(yy,y_pred)
    errors.append(np.sqrt(mse))
    r2 = r2_score(yy,y_pred)
    scores.append(r2)
    print("SCORE: {0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}"
   .format(score,  mse,np.sqrt(mse)))
    #print(r2_score(yy,y_pred))
    print(f'r2_score - {r2}')


plt.close("all")
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)


ax1.plot(qrange,scores)
ax1.set_xlabel('Alpha (Polynomial range)')
ax1.set_ylabel('Beta (Score)')
ax1.set_title('Model Score vs Polynomial Value')
ax1.axis('tight')

ax2.errorbar(qrange, mean_error, yerr=std_error)
ax2.set_xticks(qrange)
ax2.set_xlabel("ploynomial value")
ax2.set_ylabel("error")
ax2.set_title("Error vs polynomial")
ax2.axis("tight")

ax3 = plt.gca()
ax3.ticklabel_format(useOffset=False)
errors = [round(num, 5) for num in errors]
ax3.plot(qrange, errors)
ax3.set_xlabel("ploynomial value")
ax3.set_ylabel("error")
ax3.set_title("Error vs polynomial")
ax3.axis("tight")
fig.show()

#fig.show()
plt.show()

C:\Users\rkhna\AppData\Local\Temp/ipykernel_7940/93863897.py:16: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


1
10 0.05
SCORE: 0.561, MSE:20.21, RMSE:4.50
r2_score - 0.5533949449533094
2
10 0.05


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 34971.593982766106, tolerance: 3308.7880739514367
  model = cd_fast.enet_coordinate_descent(
C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6970.087154486885, tolerance: 703.8293562231763
  model = cd_fast.enet_coordinate_descent(
C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective 

SCORE: 0.780, MSE:10.03, RMSE:3.17
r2_score - 0.7783553445866203
3
10 0.05


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18416.042035377268, tolerance: 3308.7880739514367
  model = cd_fast.enet_coordinate_descent(
C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3143.427686776656, tolerance: 703.8293562231763
  model = cd_fast.enet_coordinate_descent(
C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective 

SCORE: 0.888, MSE:4.89, RMSE:2.21
r2_score - 0.8920160687898175
4
10 0.05


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8615.584275626512, tolerance: 3308.7880739514367
  model = cd_fast.enet_coordinate_descent(
C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1209.3208157896586, tolerance: 703.8293562231763
  model = cd_fast.enet_coordinate_descent(
C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective 

SCORE: 0.937, MSE:2.35, RMSE:1.53
r2_score - 0.9480080674770516
